# ファインチューニングの実装

## DatasetとDataLoaderを作成

In [1]:
#パッケージのインポート
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models,transforms

In [2]:
#1.3節で作成したクラスをインポート
from utils.dataloader_image_classification import ImageTransform,make_datapath_list,HymenopteraDataset

In [3]:
#1.3節で作成したクラスをインポート
#import sys
#sys.path.append("../1_image_classification/")
#from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

In [3]:
#アリとハチの画像へのファイルパスのリストを作成する
train_list=make_datapath_list(phase="train")
val_list=make_datapath_list(phase="val")

../1_image_classification/data/hymenoptera_data/train/**/*.jpg
../1_image_classification/data/hymenoptera_data/val/**/*.jpg


In [4]:
#Datasetを作成する
size=224
mean=(0.485,0.456,0.406)
std=(0.229,0.224,0.225)
train_dataset=HymenopteraDataset(
    file_list=train_list,transform=ImageTransform(size,mean,std),phase="train")
val_dataset=HymenopteraDataset(
    file_list=val_list,transform=ImageTransform(size,mean,std),phase="val")

In [5]:
#DataLoaderを作成する
batch_size=32

train_dataloader=torch.utils.data.DataLoader(
    train_dataset,batch_size=batch_size,shuffle=True)
val_dataloader=torch.utils.data.DataLoader(
    val_dataset,batch_size=batch_size,shuffle=False)

#辞書型オブジェクトにまとめる
dataloaders_dict={"train":train_dataloader,"val":val_dataloader}

## ネットワークモデルを作成

In [6]:
#学習済みVGG-16モデルをロード
use_pretrained=True #学習済みのパラメータを使用
net=models.vgg16(pretrained=use_pretrained)

#VGG16の最後の出力層の出力ユニットをアリとハチに付け替える
net.classifier[6]=nn.Linear(in_features=4096, out_features=2)

#訓練モードに設定
net.train()

print("ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました")



ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました


## 損失関数を定義

In [7]:
criterion=nn.CrossEntropyLoss()

## 最適化手法を設定

In [13]:
#ファインチューニングで学習させるパラメータを、変数params_to_updateの1-3に格納する
params_to_update1=[]
params_to_update2=[]
params_to_update3=[]

#学習させる層のパラメータ名を指定
update_param_names_1=["features"]
update_param_names_2=["classifier.0.weight","classifier.0.bias", 
                 "classifier.3.weight", "classifier.3.bias"]
update_param_names_3=["classifier.6.weight","classifier.6.bias"]

#パラメータごとに各リストに格納させる
for name,param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad=True
        params_to_update1.append(param)
        print("params_to_update1に格納:",name)
    elif name in update_param_names_2:
        param.requires_grad=True
        params_to_update2.append(param)
        print("params_to_update2に格納:",name)
    elif name in update_param_names_3:
        param.requires_grad=True
        params_to_update3.append(param)
        print("params_to_update3に格納:",name)
    else:
        param.requires_grad=False
        print("勾配計算なし。学習しない：",name)
        


params_to_update1に格納: features.0.weight
params_to_update1に格納: features.0.bias
params_to_update1に格納: features.2.weight
params_to_update1に格納: features.2.bias
params_to_update1に格納: features.5.weight
params_to_update1に格納: features.5.bias
params_to_update1に格納: features.7.weight
params_to_update1に格納: features.7.bias
params_to_update1に格納: features.10.weight
params_to_update1に格納: features.10.bias
params_to_update1に格納: features.12.weight
params_to_update1に格納: features.12.bias
params_to_update1に格納: features.14.weight
params_to_update1に格納: features.14.bias
params_to_update1に格納: features.17.weight
params_to_update1に格納: features.17.bias
params_to_update1に格納: features.19.weight
params_to_update1に格納: features.19.bias
params_to_update1に格納: features.21.weight
params_to_update1に格納: features.21.bias
params_to_update1に格納: features.24.weight
params_to_update1に格納: features.24.bias
params_to_update1に格納: features.26.weight
params_to_update1に格納: features.26.bias
params_to_update1に格納: features.28.weight
params_

In [14]:
#最適化手法の設定
optimizer=optim.SGD([
    {"params":params_to_update1,"lr":1e-4},
    {"params":params_to_update2,"lr":5e-4},
    {"params":params_to_update3,"lr":1e-3}],momentum=0.9)

## 学習・検証を実施

In [15]:
#モデルを学習させる関数を作成
def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):
     #初期設定
     #GPUが使えるか確認
     device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
     print("使用デバイス：",device)

     #ネットワークをGPUへ
     net.to(device)

     #ネットワークがある程度固定であれば、高速化させる
     torch.backends.cudnn.benchmark=True

    ####################

     #epochのループ
     for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch+1, num_epochs))
        print("------------------")

        #eopchごとの訓練と検証のループ
        for phase in ["train","val"]:
            if phase=="train":
                net.train() #モデルを訓練モードに
            else:
                net.eval() #モデルを検証モードに

            epoch_loss=0.0 #epochの損失和
            epoch_corrects=0 #epochの正解数

            #未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch==0) and (phase=="train"):
                continue

            #データローダーからミニバッチを取り出すループ
            for inputs,labels in tqdm(dataloaders_dict[phase]):

                #GPUが使えるならGPUにデータを送る
                inputs=inputs.to(device)
                labels=labels.to(device)

                #optimizerの初期化
                optimizer.zero_grad()

                #順伝搬(forward計算)
                with torch.set_grad_enabled(phase=="train"):
                    outputs=net(inputs)
                    loss=criterion(outputs, labels) #損失を計算
                    _,preds=torch.max(outputs,1) #ラベル

                #訓練時はバックプロパゲーション
                if phase == "train":
                    loss.backward()
                    optimizer.step()

                #結果の計算
                epoch_loss+=loss.item()*inputs.size(0) #lossの合計を更新
                #正解数の合計を更新
                epoch_corrects+=torch.sum(preds==labels.data)

            #epochごとのlossと正解率を表示
            epoch_loss=epoch_loss/len(dataloaders_dict[phase].dataset)
            epoch_acc=epoch_corrects.double()/len(dataloaders_dict[phase].dataset)

            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))



In [ ]:
#学習・検証を実行する
num_epochs=2
train_model(net,dataloaders_dict,criterion,optimizer,num_epochs=num_epochs)

## 以下落書き

In [16]:
len(dataloaders_dict["train"].dataset)

243

In [ ]:
net.named_parameters

In [ ]:
net.features